# Numpy

## Лабораторная работа №1.
Выполнил: Студент группы ДПИ22-1 Сизов Дмитрий

Замечание: при решении данных задач не подразумевается использования циклов или генераторов Python, если в задании не сказано обратного. Решение должно опираться на использования функционала библиотеки `numpy`.

1. Файл `minutes_n_ingredients.csv` содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива `numpy` типа `int32`, используя `np.loadtxt`. Выведите на экран первые 5 строк массива.

In [114]:
import numpy as np

data = np.loadtxt('data/minutes_n_ingredients.csv', np.int32, skiprows=1, delimiter=',')
print(data[:5])

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 [ 67660      5      6]
 [157911     60     14]]


2. Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [115]:
cols = [data[:, 1], data[:, 2]]
for i, col in enumerate(cols, start=1):
    print(f'| Столбец: {i}')
    print(f'Среднее: {np.mean(col)}')
    print(f'Минимум: {np.min(col)}')
    print(f'Максимум: {np.max(col)}')
    print(f'Медиана: {np.median(col)}')

| Столбец: 1
Среднее: 21601.00169
Минимум: 0
Максимум: 2147483647
Медиана: 40.0
| Столбец: 2
Среднее: 9.05528
Минимум: 1
Максимум: 39
Медиана: 9.0


3. Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля $q_{0.75}$. 

In [116]:
print(np.quantile(cols[0], 0.75))

65.0


4. Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [117]:
print(np.count_nonzero(cols[0] == 0))

cols[0][cols[0] == 0] = 1

479


5. Посчитайте, сколько уникальных рецептов находится в датасете.

In [118]:
print(len(np.unique(data[:, 0])))

100000


6. Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [119]:
print(np.sum(cols[1]))
print(len(np.unique(cols[1])))

905528
37


7. Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [120]:
print(data[np.where(data[:, 2] < 5)])

[[204134      5      3]
 [ 25623      6      4]
 [484223     18      4]
 ...
 [227056     30      4]
 [128811     15      4]
 [370915      5      4]]


8. Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [149]:
data1 = np.append(data, np.zeros((len(data), 1)), axis=1)
data1[:, 3] = data1[:, 2] / data1[:, 1]
print(data1)
print(np.max(data1[:, 3]))

24.0


9. Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [134]:
print(np.mean(data[data[:, 1].argsort()][::-1][:100, 2]))

6.61


10. Выберите случайным образом и выведите информацию о 10 различных рецептах

In [139]:
print(data[np.random.choice(len(data), 10, replace=False)])

[[278096     30     10]
 [ 14113    150     12]
 [406650     26     13]
 [ 66325     20      5]
 [217191     30      8]
 [ 81182     15      9]
 [295709     75      7]
 [272576     35      3]
 [ 74954     85     19]
 [205459   1485     13]]


11. Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [147]:
print(len(np.where(data[:, 2] < np.mean(data[:, 2]))[0]) / len(data) * 100)

58.802


12. Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.

In [152]:
data2 = np.append(data, np.zeros((len(data), 1)), axis=1).astype(np.int32)
data2[:, 3][(data2[:, 1] < 20) & (data2[:, 2] < 5)] = 1
print(data2)

[[127244     60     16      0]
 [ 23891     25      7      0]
 [ 94746     10      6      0]
 ...
 [498432     65     15      0]
 [370915      5      4      1]
 [ 81993    140     14      0]]


13. Выведите процент "простых" рецептов в датасете

In [155]:
print(len(data2[:, 3][data2[:, 3] == 1]) / len(data2) * 100)

5.093


14. Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.

In [212]:
short = data[np.where((data[:, 1] < 10))[0]].copy()
standart = data[np.where((data[:, 1] > 10) & (data[:, 1] < 20))[0]].copy()
long = data[np.where((data[:, 1] > 20))[0]].copy()
ln = min(len(short), len(standart), len(long))
data3 = np.array([short[:ln], standart[:ln], long[:ln]], np.int32)
print(data3.shape)

(3, 7588, 3)
